In [27]:
import os
import angr
import torch
from sentence_transformers import SentenceTransformer
from torch_geometric.data import Data
from sklearn.model_selection import train_test_split  # Added import for data splitting

binary_dir = './binaries'
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

data_list = []

# Use a code-aware embedding model if available
model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')  # Can replace with a code-specific model
model = model.to(device) 

def preprocessBinary(binary_file:str, isMalware: bool) -> Data:
    if binary_file.endswith('.exe'):
        print(f"Processing executable {binary_file}")
        angr_project = angr.Project(binary_file, auto_load_libs=False)
        cfg = angr_project.analyses.CFGFast() 

        functions = list(angr_project.kb.functions.values())
        function_addr_to_index = {function.addr: idx for idx, function in enumerate(functions)}  # Simplified mapping

        nodes = []
        for function in functions:
            # Extract function features
            # For example, get instruction mnemonics
            instructions = []
            for block in function.blocks:
                capstone_block = block.capstone
                for insn in capstone_block.insns:
                    instructions.append(insn.mnemonic)
            instruction_sequence = ' '.join(instructions)

            # Use instruction sequence or function name for embedding
            embedding = model.encode(instruction_sequence)
            nodes.append(embedding)

        # Build edges based on function calls using the call graph
        edge_index = []
        callgraph = angr_project.kb.callgraph  # Use the call graph from the knowledge base
        for src_addr, dst_addr in callgraph.edges():
            src_idx = function_addr_to_index.get(src_addr)
            dst_idx = function_addr_to_index.get(dst_addr)
            if src_idx is not None and dst_idx is not None:
                edge_index.append([src_idx, dst_idx])

        node_embeddings = torch.tensor(nodes, dtype=torch.float)
        if len(edge_index) == 0:
            edge_index = torch.empty((2, 0), dtype=torch.long)
        else:
            edge_index = torch.tensor(edge_index, dtype=torch.long).t().contiguous()

        label = 1 if isMalware else 0
        print(label)
        data = Data(x=node_embeddings, edge_index=edge_index, y=torch.tensor([label], dtype=torch.long))
        return data

for binary_file in os.listdir(f"{binary_dir}/malware"):
    data = preprocessBinary(f"{binary_dir}/malware/{binary_file}", True)
    data_list.append(data)

for binary_file in os.listdir(f"{binary_dir}/benign"):
    data = preprocessBinary(f"{binary_dir}/benign/{binary_file}", False)
    data_list.append(data)


# Split data into train and test sets
train_data, test_data = train_test_split(
    data_list, test_size=0.2, random_state=42,
    stratify=[d.y.item() for d in data_list]
)  # Added data splitting

cpu


c:\Development\Projects\control-flow-analysis\.venv\Lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Processing executable ./binaries/malware/crashpad_handler.exe


ERROR    | 2024-09-30 10:23:04,089 | angr.analyses.propagator.engine_vex.SimEnginePropagatorVEX | Unsupported Unop Iop_GetMSBs8x16.
ERROR    | 2024-09-30 10:23:04,096 | angr.analyses.propagator.engine_vex.SimEnginePropagatorVEX | Unsupported Unop Iop_GetMSBs8x16.
ERROR    | 2024-09-30 10:23:04,106 | angr.analyses.propagator.engine_vex.SimEnginePropagatorVEX | Unsupported Unop Iop_GetMSBs8x16.
ERROR    | 2024-09-30 10:23:04,110 | angr.analyses.propagator.engine_vex.SimEnginePropagatorVEX | Unsupported Unop Iop_GetMSBs8x16.
ERROR    | 2024-09-30 10:23:04,111 | angr.analyses.propagator.engine_vex.SimEnginePropagatorVEX | Unsupported Unop Iop_GetMSBs8x16.
ERROR    | 2024-09-30 10:23:04,200 | angr.analyses.cfg.indirect_jump_resolvers.jumptable.JumpTableProcessor | Unsupported Unop Iop_GetMSBs8x16.
ERROR    | 2024-09-30 10:23:04,274 | angr.analyses.propagator.engine_vex.SimEnginePropagatorVEX | Unsupported Unop Iop_GetMSBs8x16.
ERROR    | 2024-09-30 10:23:04,282 | angr.analyses.propagator.en

1
Processing executable ./binaries/malware/game.exe
1
Processing executable ./binaries/malware/Stacklands.exe
1
Processing executable ./binaries/benign/filmora_setup_full846.exe


...

0
Processing executable ./binaries/benign/Firefox Installer.exe


ERROR    | 2024-09-30 10:27:02,195 | angr.analyses.propagator.engine_vex.SimEnginePropagatorVEX | Unsupported statement type CAS.
WARNING  | 2024-09-30 10:27:02,490 | angr.engines.vex.claripy.ccall | Operation SBB with condition CondS is not supported in pc_calculate_condition_simple(). Consider implementing.
WARNING  | 2024-09-30 10:27:02,513 | angr.engines.vex.claripy.ccall | Operation SBB with condition CondS is not supported in pc_calculate_condition_simple(). Consider implementing.
WARNING  | 2024-09-30 10:27:02,535 | angr.engines.vex.claripy.ccall | Operation SBB with condition CondS is not supported in pc_calculate_condition_simple(). Consider implementing.
ERROR    | 2024-09-30 10:27:02,854 | angr.analyses.propagator.engine_vex.SimEnginePropagatorVEX | Unsupported statement type CAS.
ERROR    | 2024-09-30 10:27:02,856 | angr.analyses.propagator.engine_vex.SimEnginePropagatorVEX | Unsupported statement type CAS.
ERROR    | 2024-09-30 10:27:02,857 | angr.analyses.propagator.engin

0
Processing executable ./binaries/benign/VisualStudioSetup.exe


ERROR    | 2024-09-30 10:27:40,865 | angr.analyses.propagator.engine_vex.SimEnginePropagatorVEX | Unsupported statement type CAS.
ERROR    | 2024-09-30 10:27:45,211 | angr.analyses.propagator.engine_vex.SimEnginePropagatorVEX | Unsupported statement type CAS.


0


In [25]:
train_data

[Data(x=[2696, 384], edge_index=[2, 438], y=[1]),
 Data(x=[453, 384], edge_index=[2, 798], y=[1]),
 Data(x=[10865, 384], edge_index=[2, 27960], y=[1]),
 Data(x=[455, 384], edge_index=[2, 800], y=[1])]

In [26]:
test_data

[Data(x=[2524, 384], edge_index=[2, 7829], y=[1]),
 Data(x=[3155, 384], edge_index=[2, 4358], y=[1])]

In [28]:
import torch
import torch.nn.functional as F
from torch_geometric.nn import GINConv, global_add_pool

class GIN(torch.nn.Module):
    def __init__(self):
        super(GIN, self).__init__()
        self.conv1 = GINConv(
            torch.nn.Sequential(
                torch.nn.Linear(384, 128),  # Changed input dimension to match embedding size and output to 128
                torch.nn.ReLU(),
                torch.nn.Linear(128, 128)
            )
        )
        self.conv2 = GINConv(
            torch.nn.Sequential(
                torch.nn.Linear(128, 128),
                torch.nn.ReLU(),
                torch.nn.Linear(128, 128)
            )
        )
        self.conv3 = GINConv(  # Added an additional convolutional layer
            torch.nn.Sequential(
                torch.nn.Linear(128, 128),
                torch.nn.ReLU(),
                torch.nn.Linear(128, 128)
            )
        )
        self.fc1 = torch.nn.Linear(128, 64)  # Adjusted dimensions
        self.fc2 = torch.nn.Linear(64, 2)  # Output remains 2 for binary classification

    def forward(self, data):
        x, edge_index, batch = data.x, data.edge_index, data.batch  # Added batch extraction
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = self.conv2(x, edge_index)
        x = F.relu(x)
        x = self.conv3(x, edge_index)  # Added an additional layer
        x = F.relu(x)

        x = global_add_pool(x, batch)
        
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)

model = GIN()


In [29]:
from torch_geometric.loader import DataLoader
from sklearn.metrics import classification_report  # Added import for evaluation

train_loader = DataLoader(train_data, batch_size=32, shuffle=True)  # Changed to use train_data
test_loader = DataLoader(test_data, batch_size=32, shuffle=False)  # Added test_loader

optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
loss_fn = torch.nn.CrossEntropyLoss()

epochs = 10

for epoch in range(epochs):
    model.train()  # Set model to training mode
    for batch in train_loader:
        optimizer.zero_grad()
        output = model(batch)
        loss = loss_fn(output, batch.y)
        loss.backward()
        optimizer.step()
    # Evaluate on test set after each epoch
    model.eval()
    all_preds = []
    all_labels = []
    with torch.no_grad():
        for batch in test_loader:
            output = model(batch)
            preds = output.argmax(dim=1)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(batch.y.cpu().numpy())
    print(f"Epoch {epoch+1}/{epochs}")
    print(classification_report(all_labels, all_preds))  # Added evaluation


Epoch 1/10
              precision    recall  f1-score   support

           0       0.50      1.00      0.67         1
           1       0.00      0.00      0.00         1

    accuracy                           0.50         2
   macro avg       0.25      0.50      0.33         2
weighted avg       0.25      0.50      0.33         2

Epoch 2/10
              precision    recall  f1-score   support

           0       1.00      1.00      1.00         1
           1       1.00      1.00      1.00         1

    accuracy                           1.00         2
   macro avg       1.00      1.00      1.00         2
weighted avg       1.00      1.00      1.00         2



c:\Development\Projects\control-flow-analysis\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Development\Projects\control-flow-analysis\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Development\Projects\control-flow-analysis\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f

Epoch 3/10
              precision    recall  f1-score   support

           0       0.00      0.00      0.00         1
           1       0.50      1.00      0.67         1

    accuracy                           0.50         2
   macro avg       0.25      0.50      0.33         2
weighted avg       0.25      0.50      0.33         2

Epoch 4/10
              precision    recall  f1-score   support

           0       1.00      1.00      1.00         1
           1       1.00      1.00      1.00         1

    accuracy                           1.00         2
   macro avg       1.00      1.00      1.00         2
weighted avg       1.00      1.00      1.00         2

Epoch 5/10
              precision    recall  f1-score   support

           0       0.50      1.00      0.67         1
           1       0.00      0.00      0.00         1

    accuracy                           0.50         2
   macro avg       0.25      0.50      0.33         2
weighted avg       0.25      0.50      0.

c:\Development\Projects\control-flow-analysis\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Development\Projects\control-flow-analysis\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Development\Projects\control-flow-analysis\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f

Epoch 6/10
              precision    recall  f1-score   support

           0       0.50      1.00      0.67         1
           1       0.00      0.00      0.00         1

    accuracy                           0.50         2
   macro avg       0.25      0.50      0.33         2
weighted avg       0.25      0.50      0.33         2

Epoch 7/10
              precision    recall  f1-score   support

           0       0.50      1.00      0.67         1
           1       0.00      0.00      0.00         1

    accuracy                           0.50         2
   macro avg       0.25      0.50      0.33         2
weighted avg       0.25      0.50      0.33         2



c:\Development\Projects\control-flow-analysis\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Development\Projects\control-flow-analysis\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Development\Projects\control-flow-analysis\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f

Epoch 8/10
              precision    recall  f1-score   support

           0       0.50      1.00      0.67         1
           1       0.00      0.00      0.00         1

    accuracy                           0.50         2
   macro avg       0.25      0.50      0.33         2
weighted avg       0.25      0.50      0.33         2

Epoch 9/10
              precision    recall  f1-score   support

           0       0.50      1.00      0.67         1
           1       0.00      0.00      0.00         1

    accuracy                           0.50         2
   macro avg       0.25      0.50      0.33         2
weighted avg       0.25      0.50      0.33         2

Epoch 10/10
              precision    recall  f1-score   support

           0       0.50      1.00      0.67         1
           1       0.00      0.00      0.00         1

    accuracy                           0.50         2
   macro avg       0.25      0.50      0.33         2
weighted avg       0.25      0.50      0

c:\Development\Projects\control-flow-analysis\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Development\Projects\control-flow-analysis\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Development\Projects\control-flow-analysis\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f

In [30]:
# Save the model weights after training
model_path = './gin_model_weights.pth'
torch.save(model.state_dict(), model_path)
print(f"Model weights saved to {model_path}")


Model weights saved to ./gin_model_weights.pth


In [35]:
import os
import angr
import torch
from sentence_transformers import SentenceTransformer
from torch_geometric.data import Data

binary_file_path = './binaries/malware/game.exe'  # Path to the new binary file

# Load the embedding model
embedding_model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

# Load the pre-trained GIN model
model = GIN()
model.load_state_dict(torch.load('./gin_model_weights.pth'))  # Load the weights
model.eval()

# Process the new binary file
angr_project = angr.Project(binary_file_path, auto_load_libs=False)
cfg = angr_project.analyses.CFGFast()

functions = list(angr_project.kb.functions.values())
function_addr_to_index = {function.addr: idx for idx, function in enumerate(functions)}

nodes = []
for function in functions:
    instructions = []
    for block in function.blocks:
        capstone_block = block.capstone
        for insn in capstone_block.insns:
            instructions.append(insn.mnemonic)
    instruction_sequence = ' '.join(instructions)
    
    # Generate the embedding using the SentenceTransformer model
    embedding = embedding_model.encode(instruction_sequence)
    nodes.append(embedding)

# Create edge_index based on function calls
edge_index = []
callgraph = angr_project.kb.callgraph
for src_addr, dst_addr in callgraph.edges():
    src_idx = function_addr_to_index.get(src_addr)
    dst_idx = function_addr_to_index.get(dst_addr)
    if src_idx is not None and dst_idx is not None:
        edge_index.append([src_idx, dst_idx])

node_embeddings = torch.tensor(nodes, dtype=torch.float)
edge_index = torch.tensor(edge_index, dtype=torch.long).t().contiguous() if edge_index else torch.empty((2, 0), dtype=torch.long)
new_data = Data(x=node_embeddings, edge_index=edge_index)

new_data.batch = torch.zeros(new_data.x.size(0), dtype=torch.long)

model = model.to(device)
new_data = new_data.to(device)

with torch.no_grad():
    output = model(new_data)
    prediction = output.argmax(dim=1).item()  # Get the predicted class (0 or 1)
    print(f"Prediction for the binary: {'Malware' if prediction == 1 else 'Benign'}")


C:\Users\noahg\AppData\Local\Temp\ipykernel_25936\4160699208.py:14: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('./gin_model_weights.pth')

Prediction for the binary: Benign
